In [35]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [80]:
DATA_PATH = '../data'
DATA_FILE_INTERVIEWS = os.path.join(DATA_PATH, 'coded_interviews.csv')

In [86]:
data = pd.read_csv(DATA_FILE_INTERVIEWS) 

In [87]:
data

,Participant_ID,Age,Domicile,Marital_Status,Children_Count,Occupation,Health_Insurance,Employment_Status,Family_Doctor,Primary_Healthcare_Use,Private_Healthcare_Use,Education_Level,Heard_of_Cervical_Cancer,Heard_of_Testing_Programs,Performed_Preventive_Tests,Pap_Smear_Frequency,Last_Pap_Smear,Heard_of_HPV_Test,Performed_HPV_Test,Pre_Test_Information_Source,Project_Information_Source,Reaction_to_Positive_Result,Consulted_with_Others,Follow_Up_Actions,Interpretation_Aid,Current_Recommendations,Sample_Collection_Method,Overall_Experience,Barriers_to_Testing,Barriers_to_Follow_Up,Perception_of_Healthcare_Services,Preventive_Healthcare_Attitude,Knowledge_Acquisition_Method,Perceived_Barriers_in_Healthcare,Impact_of_Urban_vs_Rural,Support_System,Medical_Conditions,Economic_Barriers,Transportation_Issues,Responsibility,Healthcare_Seeking_Behavior,Overall_Insight
0,2,43,Rural,Single,7,Housewife,Yes,Social Assistance,Yes,Yes,No,Seventh Grade,Yes,No,Yes,Annual,2023,Yes,No,Midwife,Doctor,Disbelief and sadness,Yes,Ongoing,No,Hospitalization,Pap smear,Overwhelming at first,Economic,Lack of specialists,Positive,Preventive approach,Internet,Financial,Rural disadvantaged,Family,Asthma,Yes,Moderate,High,Proactive,Revealing and educational
1,3,43,Urban,Married,2,Expert in European funds,Yes,Employed,Yes,Yes,Yes,College,Yes,Yes,Yes,Sporadic,November 2023,Yes,Yes,Internet,Word of mouth,Shock and panic,Yes,Visited Cluj,No,Colposcopy,Pap smear,Overwhelming at first,NaN,Not significant,Positive,Preventive approach,Online research,Ignorance,Urban advantage,Family,NaN,NaN,NaN,High,Proactive,Revealing and educational
2,4,45,Urban,Married,2,Teacher,Yes,Employed,Yes,Yes,Yes,University,Yes,Yes,Yes,Annual,June 2023,Yes,Yes,Friends,Doctor,Shock and concern,Yes,Visited gynaecologist,Yes,Pap smear,Professional collection,Welcomed and informative,NaN,Not significant,Positive,Preventive approach,Doctor consultation,Financial,Urban advantage,Family,Menopausal symptoms,NaN,NaN,High,Proactive,Revealing and informative
3,5,51,Urban,Married,1,Professor,Yes,Employed,Yes,Yes,Yes,University,Yes,No,No,NaN,NaN,Yes,Yes,Friends,Project,Surprised and panicked,Yes,Consulted family doctor,Yes,Regular check-ups,Self-sampling,Positive,NaN,NaN,Positive,Preventive approach,Internet,Medical education gap,Urban advantage,Family,NaN,NaN,NaN,High,Proactive,Revealing and educational
4,6,42,Urban,Married,1,Hairdresser,Yes,Self-employed,Yes,Yes,Yes,High School,Yes,No,Yes,Annual,November 2023,Yes,Yes,Family doctor,Project,Shock and confusion,Yes,Visited gynaecologist,No,Regular check-ups,Professional collection,Distressing and confusing,NaN,Not significant,Positive,Preventive approach,Internet,Medical education gap,Urban advantage,Family,NaN,NaN,NaN,High,Proactive,Revealing and informative
5,7,44,Rural,Married,1,Legal advisor,Yes,Employed,Yes,Yes,No,University,Yes,Yes,Yes,Biennial,2019,Yes,Yes,Colleague,Project,Disappointment and acceptance,Yes,NaN,No,Further investigation needed,Professional collection,Revealing and concerning,NaN,Not significant,Positive,Preventive approach,Media,Financial,Rural disadvantaged,Family,NaN,NaN,NaN,High,Proactive,Revealing and concerning
6,8,55,Urban,Married,1,Computer Operator,Yes,Retired due to illness,Yes,Yes,Yes,Unknown,Yes,Yes,Yes,Annual,Autumn 2023,Yes,Yes,Magazines and TV,Own initiative,Uncertainty and stress,Yes,Visited private practice,Yes,Regular check-ups,Professional collection,Uncomfortable and stressful,Financial,Distance and transportation,Limited,Preventive approach,Internet,Limited information,Urban advantage,Family,Primitive biliary cirrhosis; autoimmune thyroi...,Yes,Yes,High,Proactive,Revealing and concerning
7,9,32,Târgu-Jiu,Single,0,Secretary at a notary office,Yes,Employed,Yes,Yes,Yes,Unknown,Yes,Yes,Yes,Annual,A few months ago in 2023,Yes,Yes,First caravan,Gynaecologist,Relief and happiness,Yes,Visited Cluj for interventions,Yes,Regular check-ups,Professional collection,Uncomfortable and reassuring,Financial,Distance and transportat